# Basic

### Problems

1. ~~Tokenizer usage is not consistent~~
2. batch_encode_plus, what does it do?
3. currently the pdf text is stitched together with '\n' from each page
4. Currently the implementation is contract nodes saved on disk. In practice it should be law nodes saved on disk and contract passed as parameter of function.

### System Info

In [ ]:
#!nvcc --version
#!which pip
#!which pip3

### Chunking

In [1]:
#!pip install datasets
#!pip install ipywidgets
#!pip install llama-index
#!pip install nvidia-ml-py3
#!pip install torch torchvision torchaudio
#!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifactory-eo.corp.qbe.com/artifactory/api/pypi/gl-pypi-remote/simple
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19189 sha256=8c4a477905b04b4781793b724c25f47760d29eb183e4f4c2db2ae3d664ce23d1
  Stored in directory: /home/yan_xu_uk_qbe_com/.cache/pip/wheels/27/51/23/64e7f2a6c0515f7568fd7b07eb19b54e498ceab83b8d24ab97
Successfully built nvidia-ml-py3


In [2]:
import datasets
import gc
import math
import numpy as np
import os
import pickle
import pynvml
import re
import shutil
import torch
import transformers

In [3]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from PyPDF2 import PdfReader
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [4]:
contracts = datasets.load_dataset('pile-of-law/pile-of-law', 'atticus_contracts', split='train+validation')

Loading Dataset Infos from /home/yan_xu_uk_qbe_com/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
Overwrite dataset info from restored data version if exists.
Loading Dataset info from /home/yan_xu_uk_qbe_com/.cache/huggingface/datasets/pile-of-law___pile-of-law/atticus_contracts/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
Found cached dataset pile-of-law (/home/yan_xu_uk_qbe_com/.cache/huggingface/datasets/pile-of-law___pile-of-law/atticus_contracts/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60)
Loading Dataset info from /home/yan_xu_uk_qbe_com/.cache/huggingface/datasets/pile-of-law___pile-of-law/atticus_contracts/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60


Loading dataset shards:   0%|          | 0/68 [00:00<?, ?it/s]

In [5]:
contract_documents = [Document(text=t) for t in contracts[:1]['text']]

In [6]:
facebook_bart_large_mnli_autotokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

In [7]:
parser = SentenceSplitter(chunk_size=300, chunk_overlap=50, tokenizer=lambda x: facebook_bart_large_mnli_autotokenizer(x)['input_ids'])

In [8]:
'''
This gives a warning
Token indices sequence length is longer than the specified maximum sequence length for this model (7786 > 1024). Running this sequence through the model will result in indexing errors

This is potentially due to facebook_bart_large_mnli_autotokenizer complaining, but doesn't impact the function here.
'''
contract_nodes = parser.get_nodes_from_documents(contract_documents, show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (7786 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
### CODE TO SAVE OBJECTS ONTO DISK ###

TOP_FOLDER = TOP_FOLDER_PATH = 'pkls'
if not os.path.exists(TOP_FOLDER):
    os.mkdir(TOP_FOLDER)

In [11]:
### SET UP SUBFOLDER

PROJECT_FOLDER = "basic"
PROJECT_FOLDER_PATH = os.path.join(TOP_FOLDER_PATH, PROJECT_FOLDER)

assert os.path.exists(TOP_FOLDER_PATH), f"Top folder path {TOP_FOLDER_PATH} does not exist."
if not os.path.exists(PROJECT_FOLDER_PATH):
    os.mkdir(PROJECT_FOLDER_PATH)

### reset folder
# shutil.rmtree(PROJECT_FOLDER_PATH)
# os.mkdir(PROJECT_FOLDER_PATH)

In [38]:
### save contract nodes
SUBFOLDER = "contract-nodes"
sub_folder_path = os.path.join(PROJECT_FOLDER_PATH, SUBFOLDER)
os.makedirs(sub_folder_path, exist_ok=True)
for i, node in enumerate(contract_nodes):
    file_path = os.path.join(sub_folder_path, f"node_{i}.pkl")
    with open(file_path, 'wb') as f:
        pickle.dump(node, f, pickle.HIGHEST_PROTOCOL)

In [11]:
### verify saving

"""
loaded_nodes = [None] * len(os.listdir(sub_folder_path))
for file_name in os.listdir(sub_folder_path):
    file_path = os.path.join(sub_folder_path, file_name)
    node_idx = int(file_name[:-4].split('_')[-1])
    with open(file_path, 'rb') as f:
        node = pickle.load(f)
        print(node)
        break
        node = np.load(f, allow_pickle=True)
        loaded_nodes[node_idx] = node
"""

"\nloaded_nodes = [None] * len(os.listdir(sub_folder_path))\nfor file_name in os.listdir(sub_folder_path):\n    file_path = os.path.join(sub_folder_path, file_name)\n    node_idx = int(file_name[:-4].split('_')[-1])\n    with open(file_path, 'rb') as f:\n        node = pickle.load(f)\n        print(node)\n        break\n        node = np.load(f, allow_pickle=True)\n        loaded_nodes[node_idx] = node\n"

In [12]:
card_act_reader = PdfReader("card_act.pdf")
card_act_number_of_pages = len(card_act_reader.pages)
card_act_text = '\n'.join(page.extract_text() for page in card_act_reader.pages)
card_act_document = Document(text=card_act_text)
card_act_nodes = parser.get_nodes_from_documents([card_act_document])

### Facebook/BART-MNLI

In [13]:
# get device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
# download model and tokenizer
facebook_bart_large_mnli_automodel_for_sequence_classification = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').to(device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
contract_nodes_folder_path = os.path.join(PROJECT_FOLDER_PATH, "contract-nodes")

In [16]:
nodes_filenames_pattern = re.compile("^(node_[0-9]+.pkl)$")
contract_nodes_filenames = [filename for filename in os.listdir(contract_nodes_folder_path) if nodes_filenames_pattern.match(filename)]
contract_nodes_filenames = sorted(contract_nodes_filenames, key=lambda x: int(x.split('.')[0].split('_')[-1]))

In [17]:
contract_nodes_loaded = []
contract_nodes_folderpath = os.path.join(PROJECT_FOLDER, "contract-nodes")
for i, contract_node_filename in enumerate(contract_nodes_filenames):
    assert int(contract_node_filename.split('.')[0].split('_')[-1]) == i, ""
    contract_node_filepath = os.path.join(contract_nodes_folder_path, contract_node_filename)
    with open(contract_node_filepath, 'rb') as f:
        contract_node = pickle.load(f)
        contract_nodes_loaded.append(contract_node)

In [18]:
record = dict()

In [19]:
with torch.no_grad():
    chunk_size = 20
    law_nodes_loaded = card_act_nodes
    law_nodes_loaded_chunks = [law_nodes_loaded[i * chunk_size : min(len(law_nodes_loaded), (i + 1) * chunk_size)] for i in range(math.ceil(len(law_nodes_loaded) / chunk_size))]
    for i, contract_node in tqdm(enumerate(contract_nodes_loaded), total=len(contract_nodes_loaded)):
        for j, law_nodes_chunk in enumerate(law_nodes_loaded_chunks):
            prop = facebook_bart_large_mnli_autotokenizer.batch_encode_plus([[law_node.text, contract_node.text] for law_node in law_nodes_chunk], return_tensors='pt', padding='longest', truncation='only_first')['input_ids']
            # prop = facebook_bart_large_mnli_autotokenizer.encode(law_node.text, contract_node.text, return_tensors='pt', truncation='only_first').expand(prop.shape[0] * 20, prop.shape[1])
            prop = torch.Tensor(prop)
            prop_gpu = prop.to(device)
            logits = facebook_bart_large_mnli_automodel_for_sequence_classification(prop_gpu).logits
            probs = logits.softmax(dim=1)
            contradiction_probs = probs[:, 0]
            for k, l in enumerate(range(j * chunk_size, min(len(law_nodes_loaded_chunks), (j + 1) * chunk_size))):
                record[(i, l)] = contradiction_probs[k]
            del prop
            del prop_gpu
            del logits
            del probs
            gc.collect()
            torch.cuda.empty_cache()
        torch.cuda.empty_cache()

100%|██████████| 36/36 [08:58<00:00, 14.97s/it]


In [20]:
contradiction_score = sum(record.values()) / len(record.values())

In [21]:
print(contradiction_score)

tensor(0.3208, device='cuda:0')
